## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
!pip3 install -U tensorflow --quiet

In [237]:
tf.__version__

'2.0.0'

In [0]:
import tensorflow as tf

### Collect Data

In [239]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/prices.csv')

### Check all columns in the dataset

In [242]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(["date","symbol"],axis=1,inplace = True)

In [244]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_new = data.iloc[0:1000]

In [246]:
data_new.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [247]:
data_new['volume'] = data_new['volume'].div(1000000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [248]:
data_new.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [249]:
data_new.shape

(1000, 5)

In [250]:
data_new.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X = data_new.drop('volume', axis=1)

In [0]:
y = data_new[['volume']]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [0]:
import numpy as np

X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')


In [0]:


X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

In [256]:
X_train.shape

(750, 4)

In [257]:
y_train.shape

(750, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)


In [0]:
X_test = transformer.fit_transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [264]:


for i in range(100):
  w, b = train(X_train, y_train, w, b)
  print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)))


  

Current Loss on iteration 0 tf.Tensor(211.58992, shape=(), dtype=float32)
Current Loss on iteration 1 tf.Tensor(209.44087, shape=(), dtype=float32)
Current Loss on iteration 2 tf.Tensor(207.4603, shape=(), dtype=float32)
Current Loss on iteration 3 tf.Tensor(205.63498, shape=(), dtype=float32)
Current Loss on iteration 4 tf.Tensor(203.95279, shape=(), dtype=float32)
Current Loss on iteration 5 tf.Tensor(202.40247, shape=(), dtype=float32)
Current Loss on iteration 6 tf.Tensor(200.97366, shape=(), dtype=float32)
Current Loss on iteration 7 tf.Tensor(199.65692, shape=(), dtype=float32)
Current Loss on iteration 8 tf.Tensor(198.44339, shape=(), dtype=float32)
Current Loss on iteration 9 tf.Tensor(197.32498, shape=(), dtype=float32)
Current Loss on iteration 10 tf.Tensor(196.29427, shape=(), dtype=float32)
Current Loss on iteration 11 tf.Tensor(195.34436, shape=(), dtype=float32)
Current Loss on iteration 12 tf.Tensor(194.4689, shape=(), dtype=float32)
Current Loss on iteration 13 tf.Tenso

### Get the shapes and values of W and b

In [265]:
print('Weights:\n', w)
print('Bias:\n',b)

Weights:
 tf.Tensor(
[[1.3139986]
 [1.362716 ]
 [1.3088093]
 [1.3752323]], shape=(4, 1), dtype=float32)
Bias:
 tf.Tensor([2.6811237], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [0]:
val = prediction(X_test, w, b)

In [275]:
val[0]

<tf.Tensor: id=16709, shape=(1,), dtype=float32, numpy=array([5.36178], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/content/drive/My Drive/Iris (1).csv')
iris.head()

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris_new = pd.get_dummies(iris['Species'])
iris_new.head()

In [0]:
iris = pd.concat([iris ,iris_new],axis=1)
iris.head()

In [0]:
iris.drop(['Species'], axis =1 , inplace= True)


In [0]:
iris.head()

### Splitting the data into feature set and target set

In [0]:
X_iris = iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
X_iris

In [0]:
y_iris = iris[['Iris-setosa','Iris-versicolor', 'Iris-virginica']]
y_iris

In [0]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.25, random_state=1)

In [0]:
print(X_train_iris.shape)
print(X_test_iris.shape)
print(y_train_iris.shape)
print(y_test_iris.shape)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(1, input_shape=(4,)))
#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [0]:
model.fit(X_train_iris,y_train_iris,          
          validation_data=(X_test_iris,y_test_iris),
          epochs=100)

### Model Prediction

In [0]:

prediction = model.predict(X_test_iris)

#Print prediction
print(prediction)




In [0]:
for i in prediction:
  predicted_num = np.argmax(i)
  print(predicted_num)

### Save the Model

In [0]:
model.save('iris_v1.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?